##Classification of US quarters

In [1]:
# load data from google drive
!unzip /content/drive/MyDrive/data_quarters.zip

Archive:  /content/drive/MyDrive/data_quarters.zip
   creating: images_sorted/
   creating: images_sorted/US.Quarter.2020.Vermont.Reverse/
   creating: images_sorted/US.Quarter.2021.Alabama.Reverse/
   creating: images_sorted/US.Quarter.2005.Oregon.Reverse/
   creating: images_sorted/US.Quarter.2010.Oregon.Reverse/
   creating: images_sorted/US.Quarter.Obverse/
   creating: images_sorted/US.Quarter.2001.New York.Reverse/
   creating: images_sorted/US.Quarter.2019.Northern Mariana Islands.Reverse/
   creating: images_sorted/US.Quarter.1999.Pennsylvania.Reverse/
   creating: images_sorted/US.Quarter.2018.Rhode Island.Reverse/
   creating: images_sorted/US.Quarter.2015.Louisiana.Reverse/
   creating: images_sorted/US.Quarter.2020.U.S. Virgin Islands.Reverse/
   creating: images_sorted/US.Quarter.2015.North Carolina.Reverse/
   creating: images_sorted/US.Quarter.2015.New York.Reverse/
   creating: images_sorted/US.Quarter.2018.Georgia.Reverse/
   creating: images_sorted/US.Quarter.2007.Uta

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

import keras
import numpy as np
import os
import PIL
import PIL.Image
import pathlib
import matplotlib.pyplot as plt

from keras import backend as K




from keras.applications import resnet50


import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot



import time




In [2]:
batch_size = 32
img_height = 256
img_width =  256

In [3]:
# data generators

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2)

train_generator = datagen.flow_from_directory(
                    'images_sorted',
                    target_size=(img_height, img_width),
                    batch_size=batch_size,
                    shuffle=True,
                    seed=123,
                    subset='training')


valid_generator = datagen.flow_from_directory(
                    'images_sorted',
                    target_size=(img_height, img_width),
                    batch_size=batch_size,
                    shuffle=True,
                    seed=123,
                    subset='validation')
        





Found 1043 images belonging to 113 classes.
Found 187 images belonging to 113 classes.


In [4]:
X_train=[]
y_train=[]

train_generator.reset()

for i in range(train_generator.__len__()):
   a,b=train_generator.next()
   X_train.extend(a)
   y_train.extend(b)


X_train=np.array(X_train)
y_train=np.array(y_train)

In [5]:
X_valid=[]
y_valid=[]

valid_generator.reset()

for i in range(valid_generator.__len__()):
   a,b=valid_generator.next()
   X_valid.extend(a)
   y_valid.extend(b)


X_valid=np.array(X_valid)
y_valid=np.array(y_valid)


In [6]:
n_classes = len(y_train[0])
n_classes

113

In [7]:
y_train = np.array([y_train[i].tolist().index(1) for i in range(len(y_train))])
y_valid = np.array([y_valid[i].tolist().index(1) for i in range(len(y_valid))])

In [8]:
# Data augmentation

data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=90,
            zoom_range=0.5,
            height_shift_range=0.1,
            width_shift_range=0.1)

In [9]:
def get_pairs(batch_size,test=False):

  if test:

    X = X_valid
    y = y_valid
    
  else:

    X = X_train
    y = y_train


   

  n, img_height,img_width, c = X_train.shape

  pairs=[np.zeros((batch_size, img_height, img_width,c)) for i in range(2)]
  targets=np.zeros((batch_size))

  targets[batch_size//2:] = 1




  for i in range(batch_size):

    
    idx1 = np.random.randint(0,n)

    pairs[0][i,:,:,:] = data_augmentation.flow(X[idx1:idx1+1])[0][0] # apply data augmentation


    idx2 = np.random.randint(0,n)


    if i < batch_size//2 :
    
      while y[idx1] == y[idx2]:

        idx2 = np.random.randint(0,n)
    

    if i >= batch_size//2 :
    
      while y[idx1] != y[idx2]:

        idx2 = np.random.randint(0,n)
  

    pairs[1][i,:,:,:] = data_augmentation.flow(X[idx2:idx2+1])[0][0]  # apply data augmentation



  return pairs, targets

In [10]:
def pairs_generator(batch_size, test = False):
    while True:
        pairs, targets = get_pairs(batch_size,test = test)
        yield (pairs, targets)

In [11]:
# Siamese network

input_left = Layers.Input((img_width,img_height,3))
input_right = Layers.Input((img_width,img_height,3))


resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

base_model = tf.keras.Sequential()
base_model.add(resnet_model)
base_model.add(Layers.GlobalAveragePooling2D())

encoded_left = base_model(input_left)
encoded_right = base_model(input_right)



L1_layer = Layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_left, encoded_right])


prediction = Layers.Dense(1,activation='sigmoid')(L1_distance)

model = tf.keras.models.Model(inputs=[input_left,input_right],outputs=prediction)



model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 2048)         23587712    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 2048)         0           sequential[0][0]             

In [12]:
model.compile(loss="binary_crossentropy",optimizer = keras.optimizers.Adam(learning_rate=1e-4))


In [13]:
batch_size = 32
steps_per_epoch = X_train.shape[0]//batch_size

In [ ]:
tic = time.time()

trained = model.fit(
            pairs_generator(batch_size),
            steps_per_epoch=steps_per_epoch,
            epochs=20,
            )

toc=time.time()    
print(str(toc-tic)+ ' s')

Epoch 1/20
22/32 [===================>..........] - ETA: 10:11 - loss: 0.7190

In [ ]:
# save model
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

In [ ]:
plt.plot(trained.history['loss'],'bo--')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.xticks(np.arange(0,20,1))
plt.show()